# 8. Hough transform – line and circle detection

In [1]:
import cv2
import numpy as np
import math

road = cv2.imread("images/streetLines_01.jpg")
chess = cv2.imread("images/chessboard_02.jpg")
coins_overlap = cv2.imread("images/coins_02.jpg")
coins = cv2.imread("images/coins_01.jpg")

**a)** Compare the functionality of `HoughLines()` and `HoughLinesP()` OpenCV functions for line detection.

> - **`HoughLines()`**: detects lines in an images; returns list of detected lines (rho, theta); the lines are infinite lines (no start or end);
> - **`HoughLinesP()`**: extension of the Hough transform used for probabilistic Hough line detection; returns line segments (start point, end point);

**b)** Use `HoughLines()` to detect lines in images like those in figure 1.a and 1.b; try different parameter values; draw
the detected lines on the image.

In [2]:
road_gray = cv2.cvtColor(road, cv2.COLOR_BGR2GRAY)
road_copy = road.copy()
chess_gray = cv2.cvtColor(chess, cv2.COLOR_BGR2GRAY)
chess_copy = chess.copy()


def detect_hough_lines(source, dest, threshold):
    edges = cv2.Canny(source, 50, 200)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold, None, 0, 0)

    if lines is not None:
        for i in range(len(lines)):
            rho = lines[i][0][0]
            theta = lines[i][0][1]
            a = math.cos(theta)
            b = math.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))
            pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))
            cv2.line(dest, pt1, pt2, (0, 0, 255), 1, cv2.LINE_AA)

    return dest


# Threshold values that show best result for each image
cv2.imshow("Image 1", detect_hough_lines(road_gray, road_copy, 150))
cv2.imshow("Image 2", detect_hough_lines(chess_gray, chess_copy, 70))

cv2.waitKey(0)
cv2.destroyAllWindows()

**c)** Use `HoughLinesP()` to detect line segments in the same images that you used in the previous problem; try
different parameter values; draw the detected line segments on the image.

In [3]:
road_gray = cv2.cvtColor(road, cv2.COLOR_BGR2GRAY)
road_copy = road.copy()

chess_gray = cv2.cvtColor(chess, cv2.COLOR_BGR2GRAY)
chess_copy = chess.copy()


def detect_hough_p_lines(source, dest, threshold, min_legth, max_gap):
    edges = cv2.Canny(source, 50, 200)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold, min_legth, max_gap)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(dest, (x1, y1), (x2, y2), (0, 0, 255), 1, cv2.LINE_AA)

    return dest


# Threshold values that show best result for each image
cv2.imshow("Image 1", detect_hough_p_lines(road_gray, road_copy, 100, 50, 10))
cv2.imshow("Image 2", detect_hough_p_lines(chess_gray, chess_copy, 30, 30, 5))

cv2.waitKey(0)
cv2.destroyAllWindows()

**d)** Use `HoughCircles()` to detect the coins present in images like those in figure 1.c and 1.d (without or with
superposition among the coins).

In [27]:
coins1 = cv2.medianBlur(coins_overlap, 5)
coins1 = cv2.cvtColor(coins_overlap, cv2.COLOR_BGR2GRAY)
coins1_copy = coins_overlap.copy()

coins2 = cv2.medianBlur(coins, 5)
coins2 = cv2.cvtColor(coins, cv2.COLOR_BGR2GRAY)
coins2_copy = coins.copy()


def detect_circles(image, dest, distance, higher_t, lower_t, min_radius, max_radius):
    circles = cv2.HoughCircles(
        image,
        cv2.HOUGH_GRADIENT,
        1,
        50,
        param1=higher_t,
        param2=lower_t,
        minRadius=min_radius,
        maxRadius=max_radius,
    )

    circles = np.uint16(np.around(circles))

    for i in circles[0, :]:
        cv2.circle(dest, (i[0], i[1]), i[2], (0, 255, 0), 2)
        cv2.circle(dest, (i[0], i[1]), 2, (0, 0, 255), 3)

    return dest


cv2.imshow("Image 1", detect_circles(coins1, coins1_copy, 50, 200, 100, 100, 250))
cv2.imshow("Image 2", detect_circles(coins2, coins2_copy, 100, 250, 50, 40, 120))

cv2.waitKey(0)
cv2.destroyAllWindows()